### Adding Realistic Substructure to the Lens Mass Model ###
This notebook adds more realistic perturbers in the form of sub-halos to the mass model of the lens.
First we will import all of the relevant packages and scripts.

In [ ]:
#setting os path to import scripts
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

#importing packages
from astropy.visualization import AsinhStretch, ImageNormalize
from astropy.visualization import simple_norm
import corner
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
import numpy as np
from paltas.Configs.config_handler_catalog import ConfigHandler
import pickle
from scipy.stats import multivariate_normal
from Scripts import lens_parameters, paltas_model, metrics, network_predictions

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### 1) We will simulate a test set of 100 mock lenses with and withoutsubstructure. ###

### 1.1) Generate Sample Parameters ###
<br>
This will generate the parameters needed for generating a sample of 100 mock lens with and without substructure.

In [ ]:
f = open(module_path+'/../Data-Tables/substructure_parameters_catalog.csv', 'w+')
f.close()

#Define how many lenses will be generated in the sample
sample_num = 101

#Define how many parameters each lens has
param_num = 10

param_names = ['index','z_lens', 'gamma_md', 'theta_E_md', 'e1_md', 'e2_md', 'center_x_md', 'center_y_md', 'gamma1_md', 'gamma2_md', 'p_center_x', 'p_center_y', 'z_source', 'mag_app_source', 'R_sersic_source',
               'n_sersic_source', 'e1_source', 'e2_source', 'center_x_source', 'center_y_source', 'z_lens_light', 'mag_app_light', 'R_sersic_light', 'n_sersic_light', 'e1_light', 'e2_light', 
               'z_point_source', 'x_point_source', 'y_point_source', 'mag_app_point_source']
#Generate the parameters to be used in the sample
param_dict = lens_parameters.perturberparameters(sample_num,module_path)
#print(param_dict)

with open(module_path+'/../Data-Tables/substructure_parameters_catalog.csv', 'a') as f:
    np.savetxt(f, param_names, fmt='%s', newline=',')
    f.write('\n')
    np.savetxt(f, param_dict, fmt='%1.15f', delimiter=',')

### 1.2) Paltas Model without substructure ###

In [ ]:
path = module_path+'/'
sample_num = 101

In [ ]:
im_WoS, metadata = paltas_model.PaltasModelWoS(path,sample_num)
im_WoS = np.asarray(im_WoS)

In [ ]:
fig,axs = plt.subplots(10,10,figsize=(10,10))
norm = ImageNormalize(im_WoS[3],stretch=AsinhStretch())

n_cols = 10
for i in range(0,100):
    axs[i//n_cols,i%n_cols].imshow(im_WoS[i], norm=norm)
    axs[i//n_cols,i%n_cols].set_xticks([])
    axs[i//n_cols,i%n_cols].set_yticks([])

plt.show()
fig.savefig(module_path+'/../Images/Sub_WoS_100.png',bbox_inches='tight')

### 1.3) Paltas model with substructure ###

In [ ]:
im_WS, metadata = paltas_model.PaltasModelWS(path,sample_num)
im_WS = np.asarray(im_WS)
#plt.axis('off')
print(np.shape(im_WS))
#plt.imshow(im_WS,norm=simple_norm(im,stretch='log',min_cut=1e-6))

In [ ]:
fig,axs = plt.subplots(10,10,figsize=(10,10))
norm = ImageNormalize(im_WS[3],stretch=AsinhStretch())

n_cols = 10
for i in range(0,100):
    axs[i//n_cols,i%n_cols].imshow(im_WS[i], norm=norm)
    axs[i//n_cols,i%n_cols].set_xticks([])
    axs[i//n_cols,i%n_cols].set_yticks([])

plt.show()
fig.savefig(module_path+'/../Images/Sub_WS_100.png',bbox_inches='tight')

### 1.4) Residuals ###
<br>
Now we will plot the residuals between the lenses with and without substructure.

In [ ]:
resid_norm =mcolors.TwoSlopeNorm(vmin=-0.025,vcenter=0,vmax=0.025)
fig,axs = plt.subplots(10,10,figsize=(10,10))

n_cols = 10
for i in range(0,100):
    imris = axs[i//n_cols,i%n_cols].imshow(im_WoS[i]-im_WS[i], norm=resid_norm,cmap='bwr')
    axs[i//n_cols,i%n_cols].set_xticks([])
    axs[i//n_cols,i%n_cols].set_yticks([])

fig.colorbar(imris, ax=axs)
plt.show()
fig.savefig(module_path+'/../Images/Sub_Res_100.png',bbox_inches='tight')